In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("Data - Original Data.csv", header=1)

# Linearly interpolate population in Scotland
df['SCT_Working_Pop'] = df['SCT_Working_Pop'].interpolate()
df['rUK_Working_Pop'] = df['rUK_Working_Pop'].interpolate()
df['Year'] = df['Year'].fillna(method='ffill')
df = df.query("Year <= 2021")

defl = df["UK_Deflator (2023=100)"]

sct_pop = df["SCT_Working_Pop"]
rUK_pop = df["rUK_Working_Pop"]
UK_pop = sct_pop + rUK_pop

for var in ['Consumption', 'Output', 'Compensation_of_Employees', 'Government_spending']:
    # Adjust for inflation using UK-wide deflator
    df[f'SCT_{var}'] = df[f'SCT_{var}'] / defl * 100
    df[f'UK_{var}'] = df[f'UK_{var}'] / defl * 100

for var in ['Consumption', 'Output', 'Compensation_of_Employees', 'Government_spending']:
    # rUK = UK - Scot
    df[f'rUK_{var}'] = df[f'UK_{var}'] - df[f'SCT_{var}']

# We use historic arithmetic mean as steady state ratio
g_y = np.mean(df['SCT_Government_spending'] / df['SCT_Output'])
g_y_ruk = np.mean(df['rUK_Government_spending'] / df['rUK_Output'])

print(f'Scotland: {int(min(df["Year"]))}Q1-{int(max(df["Year"]))}Q4 inflation-adjusted government spending-to-output ratio is {g_y}')
print(f'rUK: {int(min(df["Year"]))}Q1-{int(max(df["Year"]))}Q4 inflation-adjusted government spending-to-output ratio is {g_y_ruk}')

for var in ['Consumption', 'Output', 'Compensation_of_Employees', 'Government_spending']:
    # Express terms per capita
    df[f'SCT_{var}'] = df[f'SCT_{var}'] / sct_pop
    df[f'rUK_{var}'] = df[f'UK_{var}'] / rUK_pop

# Express UK Government spending per capita
df['UK_Government_spending'] = df['UK_Government_spending'] / UK_pop

sct_pop_share = sum(df["SCT_Working_Pop"]) / (sum(UK_pop))

print(f"Scotland's share of UK's population in {int(min(df['Year']))}Q1-{int(max(df['Year']))}Q4 is: {sct_pop_share}")

# Map dataset variables to dynare endogenous variabes used for estimation
dynare_map = {
    'SCT_Output': 'y_obs',
    'SCT_Consumption': 'c_obs',
    'SCT_Compensation_of_Employees': 'w_obs',
    'SCT_Government_spending': 'g_obs',
    'rUK_Output': 'y_obs_ruk',
    'rUK_Consumption': 'c_obs_ruk',
    'rUK_Compensation_of_Employees': 'w_obs_ruk',
    'rUK_Government_spending': 'g_obs_ruk',
    'UK_Government_spending': 'g_obs_uk'
}

for var in ['Consumption', 'Output', 'Compensation_of_Employees', 'Government_spending']:
    for country in ['SCT', 'rUK']:
        # Log-transform and take first difference
        temp_diff_log = np.log(df[f'{country}_{var}']).diff()
        
        # De-mean
        df[dynare_map[f'{country}_{var}']] = temp_diff_log[1:] - np.average(temp_diff_log[1:])

# Log-transform and take first difference
temp_diff_log = np.log(df['UK_Government_spending']).diff()

# De-mean
df[dynare_map['UK_Government_spending']] = temp_diff_log[1:] - np.average(temp_diff_log[1:])

df


Scotland: 1998Q1-2021Q4 inflation-adjusted government spending-to-output ratio is 0.2563136588315356
rUK: 1998Q1-2021Q4 inflation-adjusted government spending-to-output ratio is 0.1796672623493271
Scotland's share of UK's population in 1998Q1-2021Q4 is: 0.0851089237635477


,Year,Quarter,SCT_Deflator (2018=100),SCT_Output,SCT_Consumption,SCT_Compensation_of_Employees,SCT_Working_Pop,UK_Deflator (2023=100),UK_Output,UK_Consumption,...,rUK_Government_spending,c_obs,c_obs_ruk,y_obs,y_obs_ruk,w_obs,w_obs_ruk,g_obs,g_obs_ruk,g_obs_uk
0,1998.0,1.0,65.1,0.009646,0.006226,0.004697,3281902.0,61.2685,401149.040698,256489.060447,...,0.001852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1998.0,2.0,65.4,0.009681,0.006257,0.004800,3281797.0,61.5281,403284.027948,258408.759575,...,0.001796,0.002888,0.004139,0.000865,0.001119,0.018319,0.011460,-0.020189,-0.036180,-0.036107
2,1998.0,3.0,65.3,0.009552,0.006294,0.004842,3281692.0,62.1079,404497.656498,261387.037720,...,0.001804,0.003753,0.008144,-0.016245,-0.001182,0.005345,0.003829,0.005185,-0.001311,-0.001238
3,1998.0,4.0,65.1,0.009483,0.006291,0.004841,3281587.0,62.4503,407027.668402,260703.311273,...,0.001842,-0.002549,-0.005932,-0.009959,0.002050,-0.003617,-0.005024,-0.013417,0.015081,0.015153
4,1999.0,1.0,65.5,0.009556,0.006370,0.004901,3281482.0,62.5442,409948.484432,264598.475958,...,0.001863,0.010489,0.011519,0.004850,0.002968,0.008938,0.021026,0.011340,0.005830,0.005902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2020.0,4.0,108.2,0.011315,0.006748,0.006242,3494172.0,99.4575,542609.657391,307484.101249,...,0.002850,0.004204,-0.017579,0.013538,0.007248,0.022592,0.021212,0.011051,0.010988,0.011094
92,2021.0,1.0,109.8,0.011246,0.006585,0.006198,3494517.0,100.1883,536896.024785,298013.839939,...,0.002810,-0.026467,-0.035225,-0.008895,-0.015398,-0.010411,-0.013011,0.022359,-0.019705,-0.019599
93,2021.0,2.0,108.0,0.012079,0.007480,0.006447,3494517.0,98.3042,571947.078558,335552.295833,...,0.003190,0.125378,0.116821,0.068606,0.060553,0.036014,0.042391,0.022457,0.121110,0.121048
94,2021.0,3.0,108.5,0.012279,0.007537,0.006470,3494517.0,99.2256,581861.938854,344771.913700,...,0.003158,0.005475,0.025288,0.013616,0.014498,0.000184,0.006983,-0.003241,-0.015631,-0.015692


In [26]:
# Export for dynare

dynare_sct_data_cols = ["y_obs", "c_obs", "w_obs", "g_obs"]
dynare_ruk_data_cols = ["y_obs_ruk", "c_obs_ruk", "w_obs_ruk", "g_obs_ruk"]
dynare_data = df[dynare_sct_data_cols + dynare_ruk_data_cols + ["g_obs_uk"]][1:]
dynare_data.to_csv('../DynareData.csv', index=False)

dynare_data

,y_obs,c_obs,w_obs,g_obs,y_obs_ruk,c_obs_ruk,w_obs_ruk,g_obs_ruk,g_obs_uk
1,0.000865,0.002888,0.018319,-0.020189,0.001119,0.004139,0.011460,-0.036180,-0.036107
2,-0.016245,0.003753,0.005345,0.005185,-0.001182,0.008144,0.003829,-0.001311,-0.001238
3,-0.009959,-0.002549,-0.003617,-0.013417,0.002050,-0.005932,-0.005024,0.015081,0.015153
4,0.004850,0.010489,0.008938,0.011340,0.002968,0.011519,0.021026,0.005830,0.005902
5,-0.006969,0.012236,0.022420,-0.006268,-0.000311,0.005814,0.016149,-0.014131,-0.014030
...,...,...,...,...,...,...,...,...,...
91,0.013538,0.004204,0.022592,0.011051,0.007248,-0.017579,0.021212,0.010988,0.011094
92,-0.008895,-0.026467,-0.010411,0.022359,-0.015398,-0.035225,-0.013011,-0.019705,-0.019599
93,0.068606,0.125378,0.036014,0.022457,0.060553,0.116821,0.042391,0.121110,0.121048
94,0.013616,0.005475,0.000184,-0.003241,0.014498,0.025288,0.006983,-0.015631,-0.015692


In [4]:
import scipy.io as sio
from datetime import date

# Two governments
# Extract MH draws and store them as CSV. 
mat = sio.loadmat("/Users/edvardasvabuolas/Documents/University/Dissertation/dissertation/metropolis/dissertation_mh1_blck1.mat")

mat_df = pd.DataFrame(mat['x2'])
mat_df = mat_df.iloc[:, 6:12]
mat_df.columns = ['phi_b', 'phi_g', 'rho_g', 'phi_b_f', 'phi_g_f', 'rho_g_f']
mat_df.to_csv(f"/Users/edvardasvabuolas/Documents/University/Dissertation/Output/MCMC_SCOT_RUK_{len(mat_df)}_{date.today()}.csv")
mat_df

,phi_b,phi_g,phi_b_f,phi_g_f,rho_g,rho_g_f
0,0.187995,0.081328,0.901022,0.489992,0.049207,0.910159
1,0.187995,0.081328,0.901022,0.489992,0.049207,0.910159
2,0.187995,0.081328,0.901022,0.489992,0.049207,0.910159
3,0.187995,0.081328,0.901022,0.489992,0.049207,0.910159
4,0.187995,0.081328,0.901022,0.489992,0.049207,0.910159
...,...,...,...,...,...,...
299995,0.472034,0.105688,0.904018,0.562050,0.060721,0.896002
299996,0.592438,0.094618,0.890874,0.383243,0.106396,0.895862
299997,0.592438,0.094618,0.890874,0.383243,0.106396,0.895862
299998,0.592438,0.094618,0.890874,0.383243,0.106396,0.895862


In [3]:
import scipy.io as sio
from datetime import date

# One governments
# Extract MH draws and store them as CSV. 
mat = sio.loadmat("/Users/edvardasvabuolas/Documents/University/Dissertation/dissertation/metropolis/dissertation_mh1_blck1.mat")

mat_df = pd.DataFrame(mat['x2'])
mat_df = mat_df.iloc[:, -3:]
mat_df.columns = ['phi_b_uk', 'phi_g_uk', 'rho_g_uk']
mat_df.to_csv(f"/Users/edvardasvabuolas/Documents/University/Dissertation/Output/MCMC_UK_{len(mat_df)}_{date.today()}.csv")
mat_df

,phi_b_uk,phi_g_uk,rho_g_uk
0,0.188568,0.075748,0.900717
1,0.188568,0.075748,0.900717
2,0.188568,0.075748,0.900717
3,0.188568,0.075748,0.900717
4,0.188568,0.075748,0.900717
...,...,...,...
299995,0.468474,0.052737,0.899057
299996,0.468474,0.052737,0.899057
299997,0.468474,0.052737,0.899057
299998,0.468474,0.052737,0.899057
